# This First Cell is For Colab Setup Only!

In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.5.1

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 55kB/s 
     |████████████████████████████████| 204kB 45.8MB/s 
     |████████████████████████████████| 122kB 2.8MB/s 


In [ ]:
# You also need this for collab in your console
'''
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
'''

In [ ]:
import sparknlp

In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit
import pandas as pd

In [ ]:
spark = sparknlp.start()

In [ ]:
print("Spark NLP version", sparknlp.version())
print("Apache Spark version", spark.version)

Spark NLP version 2.5.3
Apache Spark version 2.4.5

In [ ]:
# This is for Databricks load of data from S3
# Import Sample Data
file_uri = "/mnt/mnt_s3/proj_sample.csv"

entireDataset = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(file_uri) \

entireDataset.show(10, truncate=50)

+-----------+-----------+--------------+----------+--------------+--------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------+--------------------------------------------------+-----------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------+--------------------------------------------------+-----------+
 US| 18778586| RDIJS7QYB6XNR|B00EDBY7X8| 122952789| Monopoly Junior Board Game| Toys| 5| 0| 0| N| Y| Five Stars| Excellent!!!| 8/31/15|
 US| 24769659|R36ED1U38IELG8|B00D7JFOPC| 952062646|56 Pieces of Wooden Train Track Compatible with...| Toys| 5| 0| 0| N| Y| Good quality track at excellent price|Great quality wooden track (better than some ot...| 8/31/15|
 US| 44331596| R1UE3RPRGCOLD|B002LHA74O| 818126353| Super Jumbo Playing Cards by S&S Worldwide| Toys| 2| 1| 1| N| Y| Two Stars| Cards are not as big as pictured.| 8/31/15|
 US| 23310293|R298788GS6I901|B00ARPLCGY| 261944918| Barbie Doll and Fashions Barbie Gift Set| Toys| 5| 0| 0| N| Y|my daughter loved it and i liked the price and ...|my daughter loved it and i liked the price and ...| 8/31/15|
 US| 38745832| RNX4EXOBBPN5|B00UZOPOFW| 717410439|Emazing Lights eLite Flow Glow Sticks - Spinnin...| Toys| 1| 1| 1| N| Y| DONT BUY THESE!|Do not buy these! They break very fast I spun t...| 8/31/15|
 US| 13394189|R3BPETL222LMIM|B009B7F6CA| 873028700| Melissa & Doug Water Wow Coloring Book - Vehicles| Toys| 5| 0| 0| N| Y| Five Stars|Great item. Pictures pop thru and add detail as...| 8/31/15|
 US| 2749569|R3SORMPJZO3F2J|B0101EHRSM| 723424342|Big Bang Cosmic Pegasus (Pegasis) Metal 4D High...| Toys| 3| 2| 2| N| Y| Three Stars| To keep together, had to use crazy glue.| 8/31/15|
 US| 41137196|R2RDOJQ0WBZCF6|B00407S11Y| 383363775| Fun Express Insect Finger Puppets 12ct Toy| Toys| 5| 0| 0| N| Y| Five Stars| I was pleased with the product.| 8/31/15|
 US| 433677|R2B8VBEPB4YEZ7|B00FGPU7U2| 780517568|Fisher-Price Octonauts Shellington's On-The-Go ...| Toys| 5| 0| 0| N| Y| Five Stars| Children like it| 8/31/15|
 US| 1297934|R1CB783I7B0U52|B0013OY0S0| 269360126| Claw Climber Goliath/ Disney's Gargoyles| Toys| 1| 0| 1| N| Y| Shame on the seller !!!|Showed up not how it's shown . Was someone's ol...| 8/31/15|
+-----------+-----------+--------------+----------+--------------+--------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------+--------------------------------------------------+-----------+
only showing top 10 rows

In [ ]:
# This is for data load in Colab
entireDataset = spark.read.csv('/content/proj_sample.csv')

In [ ]:
entireDataset.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14']

In [ ]:
trainDataset, testDataset = entireDataset.randomSplit([0.75, 0.25], seed=12345)

In [ ]:
# Content is inside review_body
document = DocumentAssembler() \
  .setInputCol("_c13") \
  .setOutputCol("document")

In [ ]:
# Sentence detector
use = UniversalSentenceEncoder.pretrained() \
  .setInputCols(["document"]) \
  . setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
# Our labels / classes are the 1 - 5 star ratings
# These are in the star_rating col

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("_c7")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

In [ ]:
# Declare the pipeline
use_clf_pipeline = Pipeline(
  stages = [
    document,
    use,
    classifierdl
  ]
)

In [ ]:
# Begin fitting on the data
use_pipelineModel = use_clf_pipeline.fit(trainDataset)

In [ ]:
# Set predictions on testDataset
predictions = use_pipelineModel.transform(testDataset)

In [ ]:
predictions.select('_c7','_c13', 'class.result').show(10, truncate=80)

+---+--------------------------------------------------------------------------------+------+
|_c7|                                                                            _c13|result|
+---+--------------------------------------------------------------------------------+------+
|  5|                                                  Love it great add to collecton|   [5]|
|  5|These paper lanterns are adorable! The colors are bright, the patterns are fu...|   [5]|
|  1|         Showed up not how it's shown . Was someone's old toy. with paint on it.|   [5]|
|  4|i like it but i absoloutely hate that some dolls don't have pets like this on...|   [5]|
|  2|IT's OK, but not as good as the old Bite Meez puppets. This puppet is very th...|   [5]|
|  5|                     It was a birthday present for my grandson and he LOVES IT!!|   [5]|
|  1|This was to be a gift for my husband for our new pool. Did not receive the co...|   [5]|
|  5|                                     We play this game 

In [ ]:
# Import reports for quantifying results
from sklearn.metrics import classification_report, accuracy_score

df = use_pipelineModel.transform(testDataset).select('_c7','_c13', 'class.result').toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

In [ ]:
print(classification_report(df._c7, df.result))
print(accuracy_score(df._c7, df.result))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.44      1.00      0.62         4

    accuracy                           0.44         9
   macro avg       0.09      0.20      0.12         9
weighted avg       0.20      0.44      0.27         9

0.4444444444444444


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
